#### https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html
#### https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html
#### https://github.com/zjor/inverted-pendulum/blob/master/python/free-pendulum.py
#### https://riptutorial.com/matplotlib/example/23558/basic-animation-with-funcanimation

# Bewegungsgleichung freies, gedämpftes Pendel:
$ \ddot\theta(t) + \frac{d}{m \cdot l^2} \cdot \dot\theta(t) + \frac{g}{l} \cdot \sin(\theta(t)) = 0 $

### Systemgleichungen:
$ \dot\theta(t) = \omega(t) $

$ \dot\omega(t) = - \frac{d}{m \cdot l^2} \cdot \omega(t) - \frac{g}{l} \cdot \sin(\theta(t)) $

### State Vector y:

$ y(t) = \begin{vmatrix}
\theta(t)\\
\omega(t)
\end{vmatrix}$

In [ ]:
from ipywidgets import FloatSlider, interact_manual
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np

# gegebene Werte
g = 9.81
m = 1.0

# Zeitbereich
t1 = 15
frames = 200
t = np.linspace(0,t1,frames)


def pendulum(y, t, d, l):
    theta, omega = y
    return [omega, -d/(m*l**2)*omega - g/l*np.sin(theta)]

def plot_pendulum(t, sol, d, l, t0):
    %matplotlib inline
    fig, axs = plt.subplots(1, 1, figsize=(12,8), dpi=200)
    fig.suptitle('damped pendulum', fontsize=24)
    string = r"$d=%s\enspaceNm\cdot s;\enspacel=%s\enspacem;\enspace\theta_0=%s\enspacerad$"%(d,l,t0)
    axs.set_title(string, fontsize=18)
    axs.plot(t, sol[:, 0], 'b', label=r'$\theta(t)\enspace[rad]$')
    axs.plot(t, sol[:, 1], 'g', label=r'$\omega(t)\enspace([\frac{rad}{s}])$')
    axs.set_xlabel('time [s]', fontsize=20)
    axs.set_ylabel('amplitutde', fontsize=20)
    axs.legend(loc='best')
    axs.grid()
    plt.show()
    fig.savefig('pendulum.png')
    
    
damping = FloatSlider(value=0.42,min=0,max=1,step=0.01,description='$d$')
length = FloatSlider(value=1.0,min=0.5,max=3.0,step=0.5,description='$l$')
theta_0 = FloatSlider(value=3.14,min=0,max=np.pi,step=0.01,description=r'$\theta_0$')

def interaction(d, l, t0):
    
    # Anfangswerte
    y0 = [t0, 0.0] 
    
    # Integration
    sol = odeint(pendulum, y0, t, args=(d,l))
    
    # Visualisierung
    plot_pendulum(t, sol, d, l, t0)
    

interactive_plot = interact_manual(interaction, d=damping, l=length, t0=theta_0)

In [ ]:
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter
%matplotlib notebook

# Slider Werte
l = length.value
d = damping.value
t0 = theta_0.value

# ODE neu berechnen
solution = odeint(pendulum, [t0, 0.0], t, args=(d, l))

# Kartesischen Koordinaten der Pendelmasse 
x = l * np.sin(solution[:, 0])
y = - l * np.cos(solution[:, 0])

# Animations Figure vorbereiten
fig = plt.figure(figsize=(16, 5))
gs = GridSpec(nrows=1, ncols=2, width_ratios=[1, 2])
string = r"$d=%s\enspaceNm\cdot s;\enspacel=%s\enspacem;\enspace\theta_0=%s\enspacerad$"%(d,l,t0)
fig.suptitle(string, fontsize=24)

# Pendelanimation
movement = fig.add_subplot(gs[0, 0], autoscale_on=False, xlim=(-l*1.2, l*1.2), ylim=(-l*1.2, l*1.2))

# Pendelmasse
circle = plt.Circle((0,0), 0.08, fc='r', zorder=3)
circle = movement.add_patch(circle)

# Pendelstab
line, = movement.plot([], [], 'o-', color='k',lw=3)

# Zeitlabel
time_template = 'time = %.1fs'
time_text = movement.text(0.1, 0.85, '', fontsize=12, transform=movement.transAxes)

# Zeitverlauf Animation
timeline = fig.add_subplot(gs[:, 1])
timeline.plot(t, solution[:, 0], 'b', label=r'$\theta(t)\enspace[rad]$')
timeline.plot(t, solution[:, 1], 'g', label=r'$\omega(t)\enspace([\frac{rad}{s}])$')
timeline.set_xlabel('time [s]', fontsize=16)
timeline.legend(loc='best')
timeline.grid()

# Pendelmasse im Zeitverlauf
theta_point, = timeline.plot([], [], 'o-', color='r', lw=3)
omega_point, = timeline.plot([], [], 'o-', color='r', lw=3)


def animate(i):
    
    # Movement
    line.set_data([0, x[i]], [0, y[i]])  
    circle.set_center((x[i], y[i]))
    time_text.set_text(time_template % (t[i]))
    
    # Timeline
    theta_point.set_data(t[i], solution[i, 0])
    omega_point.set_data(t[i], solution[i, 1])
    return line, time_text


ani = animation.FuncAnimation(fig, animate, frames=len(t), interval=10, repeat=False)
ani.save('pendulum.gif', writer='pillow', fps=frames/t1, dpi=180)